# Projeto 2 - NLP

## Grupo:
- Laianna Lana Virginio da Silva - lvs2@cin.ufpe.br
- Lucas Natan Correia Couri - lncc2@cin.ufpe.br
- Mariama Celi Serafim de Oliveira - mcso@cin.ufpe.br

# Importando bibliotecas

In [1]:
import preprocessing as pp
import load_files as lf
import nlp_algorithms as nlp
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer

# Carregando conjunto de dados

In [2]:
pd.set_option('display.max_colwidth', -1)
## dfSourceV2 = pd.read_csv('../Dados/v2_semeval-2022_task8_train-data_batch.csv')
train_v1 = pd.read_csv('dados/train v0.1.csv')

train_v1_enen = train_v1[(train_v1['url1_lang'] == 'en') & (train_v1['url2_lang'] == 'en')]

train_v1_enen.head()

C:\Users\mcso\AppData\Local\Temp/ipykernel_13808/2099918564.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,https://web.archive.org/web/www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://web.archive.org/web/www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,https://web.archive.org/web/www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://web.archive.org/web/www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,https://web.archive.org/web/gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://web.archive.org/web/gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,https://web.archive.org/web/news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://web.archive.org/web/www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


In [3]:
data_path = 'dados/train v0.1/'

lista_docs = []
lista_error = []
lista_vazio = []
values = train_v1_enen[['pair_id', 'Overall']]

for index, values in values.iterrows():
    
    try:
        
        json_pair = lf.get_json_document_pair(data_path, values['pair_id'])
        text_doc1 = json_pair[0]['text']
        text_doc2 = json_pair[1]['text']
        
        if ( len(text_doc1) > 0 ) and ( len(text_doc2) > 0 ):
            lista_docs.append((values['pair_id'], text_doc1, text_doc2, values['Overall']))
        else:
            lista_vazio.append(values['pair_id'])
    
    except:
        lista_error.append(values['pair_id'])

In [4]:
df_text = pd.DataFrame(lista_docs,  columns=['pair_id', 'doc1', 'doc2', 'Overall'])
#df_text.head(1)

In [5]:
df_text.shape

(1357, 4)

# Pré-processamento dos dados

In [6]:
pre_processing_list = [
    {},
    {"basic_processing": True},
    {"basic_processing": True, "no_stopwords": True},
    {"basic_processing": True, "stemming": True},
    {"basic_processing": True, "no_stopwords": True, "stemming": True},
    {"basic_processing": True, "lema":  True},
    {"basic_processing": True, "no_stopwords": True, "lema": True}]

#df_pp = pp.pre_process_all(df_text, pre_processing_list)
df_pp = pp.pre_process_all(df_text.iloc[:10], pre_processing_list)

c:\Users\mcso\Desktop\residencia\NLP\projeto2-nlp-sidi-residencia\preprocessing.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"doc1_pipeline{index}"] = df["doc1"].apply(lambda x: pre_process(x, **param))
c:\Users\mcso\Desktop\residencia\NLP\projeto2-nlp-sidi-residencia\preprocessing.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"doc2_pipeline{index}"] = df["doc2"].apply(lambda x: pre_process(x, **param))


In [7]:
#df_pp.head(1)

# Algoritmos

In [8]:
df_results = df_pp[["pair_id", "Overall"]]
df_results.head(1)

,pair_id,Overall
0,1484084337_1484110209,4.0


## Bag of Words (CountVectorizer)

In [9]:
df_bow = nlp.apply_bow(df_pp, len(pre_processing_list))

In [10]:
df_results = df_results.join(df_bow)

In [11]:
df_results.head(5)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636
3,1576314516_1576455088,2.000000,0.678074,0.683262,0.461996,0.683147,0.482403,0.684565,0.475937
4,1484036253_1483894099,1.250000,0.790918,0.780586,0.479421,0.799176,0.540089,0.799223,0.535830


## TF-IDF

In [12]:
df_tf_idf = nlp.apply_tf_idf(df_pp, len(pre_processing_list))

In [13]:
df_results = df_results.join(df_tf_idf)

In [14]:
df_results.head(5)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6,tf_idf0,tf_idf1,tf_idf2,tf_idf3,tf_idf4,tf_idf5,tf_idf6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200,0.204060,0.214436,0.079230,0.220277,0.074928,0.212621,0.075596
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285,0.256470,0.253428,0.042134,0.248462,0.043084,0.249139,0.041451
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636,0.341090,0.353646,0.234897,0.370994,0.256862,0.395499,0.280169
3,1576314516_1576455088,2.000000,0.678074,0.683262,0.461996,0.683147,0.482403,0.684565,0.475937,0.472555,0.466816,0.391316,0.485522,0.416267,0.470828,0.396233
4,1484036253_1483894099,1.250000,0.790918,0.780586,0.479421,0.799176,0.540089,0.799223,0.535830,0.570528,0.522019,0.400790,0.592051,0.487107,0.582280,0.474820


## BERT

In [15]:
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
model_name = 'base_mean'

df_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)

In [16]:
model = SentenceTransformer('sentence-transformers/bert-base-nli-max-tokens')
model_name = 'base_max'

df_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)

Downloading: 100%|██████████| 391/391 [00:00<00:00, 131kB/s]
Downloading: 100%|██████████| 3.97k/3.97k [00:00<00:00, 4.12MB/s]
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 1.00kB/s]
Downloading: 100%|██████████| 229/229 [00:00<00:00, 115kB/s]
Downloading: 100%|██████████| 438M/438M [00:22<00:00, 19.6MB/s] 
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 17.7kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 56.1kB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 800kB/s]  
Downloading: 100%|██████████| 397/397 [00:00<00:00, 196kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 663kB/s] 
Downloading: 100%|██████████| 190/190 [00:00<00:00, 96.9kB/s]


In [17]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model_name = 'miniLM'

df_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)

In [18]:
df_results.head(10)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6,tf_idf0,...,bert_base_max4,bert_base_max5,bert_base_max6,bert_miniLM0,bert_miniLM1,bert_miniLM2,bert_miniLM3,bert_miniLM4,bert_miniLM5,bert_miniLM6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200,0.204060,...,0.811856,0.855984,0.875019,0.120794,0.094716,0.135753,0.178856,0.283669,0.098491,0.128578
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285,0.256470,...,0.850374,0.887332,0.887321,0.141600,0.197086,0.170077,0.292377,0.312857,0.222857,0.178241
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636,0.341090,...,0.940309,0.933166,0.953908,0.660645,0.575526,0.475314,0.535299,0.564201,0.595904,0.468557
3,1576314516_1576455088,2.000000,0.678074,0.683262,0.461996,0.683147,0.482403,0.684565,0.475937,0.472555,...,0.920493,0.900969,0.932070,0.542525,0.598751,0.804728,0.604248,0.626196,0.599509,0.794610
4,1484036253_1483894099,1.250000,0.790918,0.780586,0.479421,0.799176,0.540089,0.799223,0.535830,0.570528,...,0.953682,0.962042,0.951390,0.797832,0.831320,0.847097,0.726931,0.814400,0.823877,0.835823
5,1484189120_1484113136,2.000000,0.785585,0.777706,0.390119,0.790774,0.486256,0.780084,0.401036,0.559682,...,0.944060,0.918726,0.933383,0.640959,0.680841,0.755826,0.637493,0.707139,0.666830,0.745372
6,1484034982_1483785560,3.666667,0.485899,0.487111,0.142310,0.483234,0.145531,0.484682,0.139947,0.159859,...,0.828059,0.865151,0.839587,0.132723,0.154511,0.172354,0.193349,0.226670,0.188940,0.187399
7,1484188439_1484378177,3.333333,0.672483,0.690785,0.071027,0.678436,0.078603,0.686623,0.073578,0.300203,...,0.840234,0.830191,0.855816,0.146774,0.220597,0.354224,0.357980,0.474914,0.210272,0.329089
8,1484035244_1483938857,3.333333,0.659135,0.648809,0.222553,0.634050,0.212486,0.638550,0.211200,0.306608,...,0.879690,0.897818,0.878569,0.360761,0.436945,0.559885,0.523162,0.524565,0.458907,0.556044
9,1484396075_1483794933,2.666667,0.197386,0.199778,0.000000,0.182743,0.000000,0.182743,0.000000,0.035113,...,0.597426,0.657908,0.608690,-0.024944,-0.024424,0.066099,-0.046618,0.035567,-0.016644,0.091770


In [19]:
m = m

NameError: name 'm' is not defined

## Doc2vec

In [ ]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec
from scipy import spatial

In [ ]:
#data = join_unique_docs(dfText, 'msg_lemmatized1', 'msg_lemmatized2')
data = join_docs(dfText, 'msg_lemmatized1', 'msg_lemmatized2', unique = True)


tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)] 

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=30, min_count=0, epochs=80)

model.build_vocab(tagged_data)

model.train(tagged_data, total_examples=model.corpus_count, epochs=80)

model.save("d2v.model")

model = Doc2Vec.load("d2v.model")

def doc2vec_cos(doc1, doc2):
    infer1 = model.infer_vector(doc1)
    infer2 = model.infer_vector(doc2)
    #cos_distance = spatial.distance.cosine(infer1, infer2) #pode ser >1
    cos_similarity = 1-spatial.distance.cosine(infer1, infer2) #de 0 a 1
    return cos_similarity


dfText['doc2vec'] = dfText.apply(lambda row: doc2vec_cos(row['msg_tokenized1'], row['msg_tokenized2']), axis=1)

dfText[['doc2vec', 'Overall']].head(20)

dfText.describe()



## Word2vec

In [ ]:
import gensim.downloader as api

# print(api.load("word2vec-google-news-300", return_path=True))

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

#glove_vectors = gensim.downloader.load('glove-twitter-25')
#model = KeyedVectors.load_word2vec_format('data/wiki.en.vec', binary=False)
# model = KeyedVectors.load_word2vec_format('model/word2vec-google-news-300.gz', binary=True)
model = api.load('word2vec-google-news-300')

def get_mean_vector(word2vec_model, words): #words eh um documento inteiro
    # remove out-of-vocabulary words
    words = [word for word in words if word in model.index_to_key]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return []


get_mean_vector(model, dfText['no_stopwords1'][0])

def word2vec_cos(doc1, doc2, model):
    mean_doc1 = get_mean_vector(model, doc1)
    mean_doc2 = get_mean_vector(model, doc2)

    infer1 = mean_doc1
    infer2 =  mean_doc2
    #cos_distance = spatial.distance.cosine(infer1, infer2) #pode ser >1
    cos_similarity = 1-spatial.distance.cosine(infer1, infer2) #de 0 a 1
    return cos_similarity

dfText['word2vec_mean'] = dfText.iloc[0:10].apply(lambda row: word2vec_cos(" ".join(row['no_stopwords1']), " ".join(row['no_stopwords2']), model), axis=1)

dfText[['word2vec_mean', 'tfidf', 'Overall']].head(10)